In [1]:
%load_ext autoreload
%autoreload 2

from meri.utils import setup_logging
import logging
from functools import lru_cache
from IPython.display import display_markdown

logger = logging.getLogger(__name__)

logging.basicConfig()

In [2]:
#import wikipedia
#print(wikipedia.summary("Mathematics"))
#wikipedia.search("Mathematics")
#html = wikipedia.page("Mathematics").url

# # Convert to markdown
# from markdownify import markdownify as md
# from IPython.display import display_markdown
# md_ = md(html)
# display_markdown(md_, raw=True)

In [3]:
from datetime import datetime
import requests
import requests.adapters
from meri.extractor._processors import html_to_markdown
from haystack import Document

from meri.scraper import get_user_agent, try_setup_requests_cache

try_setup_requests_cache()

# Set request session
session = requests.Session()
retries = requests.adapters.Retry()
session.mount("https://", requests.adapters.HTTPAdapter(max_retries=retries))
session.headers.update({
    "User-Agent": get_user_agent(),
})

search_term = "Climate Change"
search_url = "https://en.wikipedia.org/w/api.php?action=query&list=search&srnamespace=0&format=json"
page_url = "https://en.wikipedia.org/?action=render"
search_response = session.get(search_url, params={
    "srsearch": search_term
})
data = search_response.json()
docs = []

def page_summary(page_ids: list[int]):
    page_url = "https://en.wikipedia.org/w/api.php?action=query&prop=extracts&format=json&redirects=1&exintro=1&explaintext=1"
    page_response = session.get(page_url, params={"pageids": "|".join(map(str, page_ids))})
    return page_response.json()

for result in data["query"]["search"]:    
    print(f"[-] Fetching article {result['title']!r}")
    if result["ns"] != 0:
        print(f"[!] Skipping non-article page {result['title']!r}")
        continue
    page_response = session.get(page_url, params={"curid": result["pageid"]})

    summary = html_to_markdown(result["snippet"])

    # Convert to markdown
    content = f"# {result['title']}\n\n" + html_to_markdown(page_response.text)
    docs.append(
        Document(
            content=content,
            meta={
                "title": result["title"],
                "url": page_response.url,
                "snippet": summary,
                "language": "en",
                "date_added": datetime.now().isoformat(),
            },
        )
    )

print(f"Collected {len(docs)} documents")

2025-04-09T09:19:45.291052Z [debug    ] Initializing backend: None /home/vscode/.cache/meri/requests-cache lineno=66 module=requests_cache.backends
DEBUG:requests_cache.backends:Initializing backend: None /home/vscode/.cache/meri/requests-cache
2025-04-09T09:19:45.292988Z [debug    ] Initialized SQLiteDict with serializer: SerializerPipeline(name=pickle, n_stages=2) lineno=322 module=requests_cache.backends.base
DEBUG:requests_cache.backends.base:Initialized SQLiteDict with serializer: SerializerPipeline(name=pickle, n_stages=2)
2025-04-09T09:19:45.294656Z [debug    ] Opening connection to /home/vscode/.cache/meri/requests-cache.sqlite:responses lineno=252 module=requests_cache.backends.sqlite
DEBUG:requests_cache.backends.sqlite:Opening connection to /home/vscode/.cache/meri/requests-cache.sqlite:responses
2025-04-09T09:19:45.298420Z [debug    ] Initialized SQLiteDict with serializer: None lineno=322 module=requests_cache.backends.base
DEBUG:requests_cache.backends.base:Initialized SQ

[-] Fetching article 'Climate change'


2025-04-09T09:19:47.896818Z [debug    ] Cache directives from request headers: CacheDirectives() lineno=103 module=requests_cache.policy.actions
DEBUG:requests_cache.policy.actions:Cache directives from request headers: CacheDirectives()
2025-04-09T09:19:47.898471Z [debug    ] Pre-read cache checks: Passed  lineno=368 module=requests_cache.policy.actions
DEBUG:requests_cache.policy.actions:Pre-read cache checks: Passed
2025-04-09T09:19:47.905180Z [debug    ] Post-read cache actions: CacheActions(expire_after=-1) lineno=217 module=requests_cache.policy.actions
DEBUG:requests_cache.policy.actions:Post-read cache actions: CacheActions(expire_after=-1)


[-] Fetching article 'Climate change denial'


2025-04-09T09:19:50.166132Z [debug    ] Cache directives from request headers: CacheDirectives() lineno=103 module=requests_cache.policy.actions
DEBUG:requests_cache.policy.actions:Cache directives from request headers: CacheDirectives()
2025-04-09T09:19:50.168560Z [debug    ] Pre-read cache checks: Passed  lineno=368 module=requests_cache.policy.actions
DEBUG:requests_cache.policy.actions:Pre-read cache checks: Passed
2025-04-09T09:19:50.175812Z [debug    ] Post-read cache actions: CacheActions(expire_after=-1) lineno=217 module=requests_cache.policy.actions
DEBUG:requests_cache.policy.actions:Post-read cache actions: CacheActions(expire_after=-1)


...

[-] Fetching article 'United Nations Climate Change Conference'


2025-04-09T09:19:57.995120Z [debug    ] Cache directives from request headers: CacheDirectives() lineno=103 module=requests_cache.policy.actions
DEBUG:requests_cache.policy.actions:Cache directives from request headers: CacheDirectives()
2025-04-09T09:19:57.997257Z [debug    ] Pre-read cache checks: Passed  lineno=368 module=requests_cache.policy.actions
DEBUG:requests_cache.policy.actions:Pre-read cache checks: Passed
2025-04-09T09:19:58.002242Z [debug    ] Post-read cache actions: CacheActions(expire_after=-1) lineno=217 module=requests_cache.policy.actions
DEBUG:requests_cache.policy.actions:Post-read cache actions: CacheActions(expire_after=-1)


[-] Fetching article 'Climate change adaptation'


2025-04-09T09:19:59.092512Z [debug    ] Cache directives from request headers: CacheDirectives() lineno=103 module=requests_cache.policy.actions
DEBUG:requests_cache.policy.actions:Cache directives from request headers: CacheDirectives()
2025-04-09T09:19:59.094893Z [debug    ] Pre-read cache checks: Passed  lineno=368 module=requests_cache.policy.actions
DEBUG:requests_cache.policy.actions:Pre-read cache checks: Passed
2025-04-09T09:19:59.099577Z [debug    ] Post-read cache actions: CacheActions(expire_after=-1) lineno=217 module=requests_cache.policy.actions
DEBUG:requests_cache.policy.actions:Post-read cache actions: CacheActions(expire_after=-1)
2025-04-09T09:19:59.119112Z [debug    ] Cache directives from request headers: CacheDirectives() lineno=103 module=requests_cache.policy.actions
DEBUG:requests_cache.policy.actions:Cache directives from request headers: CacheDirectives()
2025-04-09T09:19:59.121556Z [debug    ] Pre-read cache checks: Passed  lineno=368 module=requests_cache.p

[-] Fetching article 'Climate change (disambiguation)'
[-] Fetching article 'Intergovernmental Panel on Climate Change'
Collected 10 documents


In [4]:
SUMMARY_PROMPT = """
You are a helpful AI assistant for Klikkikuri service that summarizes text.

Your task is to summarize the given text in the <article> -section. Aim for the following:
- Summary __MUST__ be as non-opinionated as possible, and close to the original text.
- Summary __MUST__ be in the same language as the text.
- Summary __MAY NOT__ contain any additional information, context or commentary in the summary that is not in the text.
- Summary __NEEDS__ to contain the entities and relations of the text.
- Summary __NEEDS__ to contain the most important points of the text.
- Summary __SHOULD__ distill the essential points of the text.
- Summary __SHOULD__ containt the most unique points of the text of the section.
- Summary __SHOULD__ be as short as feasible.
- Summary __MAY ONLY__ include basic semantic formatting in markdown like bold or emphasis, but __MAY NOT__ include formatting like links, images, tables, headings, etc.
- Approach the task as extractive task, but format it as a abstact summary.
- Do __NOT__ add *any* preceding sentences (like "This is a summary of ...", "The section ...") or trailing sentences.
- For sources or references, return `{{SKIP_TAG}}`.
- The output should begin immediately with the summarized content.
- If no sensible summary can be generated, return `{{SKIP_TAG}}`.

You are to summarize following wikipedia article titled {{article_title|escape}}.
Section of the article to summarize is {{section_title|escape}} – keep the summary relevant to it.
Subsectioned content inside of `<summary>` -tags are previously summarized sections for context – focus on providing additional information to them, but do not repeat content that is provided already on them.

Article to summarize follows in markdown format:
<article>
{{text}}
</article>

Now please summarize the text in the <article> -section. 
You MUST only produce the summary, no yapping, no explanations.
"""

from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List
from haystack import Pipeline, component
from haystack.components.preprocessors import DocumentCleaner
from haystack.components.generators import OpenAIGenerator
from haystack.components.builders import PromptBuilder
from meri.llm import get_generator
from meri.wp import BetterMarkdownChunker


def reduce_to_summary(md: str, lang="en"):
    MAX_CONCURRENT_WORKERS: int = 3
    SKIP_TAG = "<skip>"
    doc_tree = BetterMarkdownChunker(md, language=lang).parse()

    prompt_builder = PromptBuilder(SUMMARY_PROMPT)
    llm = get_generator()

    p = Pipeline(max_runs_per_component=1)
    p.add_component("prompt_builder", prompt_builder)
    p.add_component("llm", llm)
    p.connect("prompt_builder", "llm")

    def make_node_text(node, titles):
        section_titles = titles + [node['title']]
        section_titles_md = "\n\n".join(f"#{'#' * i} {_title}" for i, _title in enumerate(section_titles))
        return f"{section_titles_md}\n\n{node['body']}", section_titles

    @lru_cache(maxsize=128)
    def summarize_node_sync(text, titles) -> str:
        results = p.run({
            "prompt_builder": {
                "text": text,
                "article_title": titles[0],
                "section_title": titles[-1],
                "SKIP_TAG": SKIP_TAG,
            },
        })
        if "llm" in results:
            return results["llm"]["replies"][0]
        raise RuntimeError("No summary generated")

    def generate_summary(node, titles, executor, futures_map):
        print(f" |{'-'*len(titles)} node: {node['title']}")
        new_titles = titles + [node["title"]]
        node_text, full_titles = make_node_text(node, titles)

        if node['children']:
            # Recurse first (sequentially), collect child summaries later
            for child in node['children']:
                generate_summary(child, new_titles, executor, futures_map)

            # After all children are scheduled, combine summaries
            subsection_summaries = []
            for child in node['children']:
                child_summary = child.get("summary")
                subsection_summaries.append(child_summary)

            print(f" >{'-'*len(titles)} {len(subsection_summaries)} summaries for section: {node['title']}")
            sections = [node_text]
            subsection_titles = (f"#{'#' * (len(new_titles) + 1)} {_title}" for _title in titles[1:])
            for title, summary in zip(subsection_titles, subsection_summaries):
                if summary.strip() == SKIP_TAG:
                    logger.debug(f"Skipping summary for {title} due to SKIP_TAG")
                    continue

                sections += [f"{title}\n\n<summary>{summary}</summary>"]
            node_text = "\n\n".join(sections)

        # Submit summarization task
        future = executor.submit(summarize_node_sync, node_text, tuple(new_titles))
        futures_map[future] = node

    with ThreadPoolExecutor(max_workers=MAX_CONCURRENT_WORKERS) as executor:
        futures_map = {}
        generate_summary(doc_tree, [], executor, futures_map)

        for future in as_completed(futures_map):
            node = futures_map[future]
            node['summary'] = future.result()

    display(doc_tree)

    return doc_tree['summary']

display_markdown(reduce_to_summary(docs[0].content, lang="en"), raw=True)


2025-04-09T09:20:00.079298Z [debug    ] Registering <class 'haystack.components.preprocessors.document_cleaner.DocumentCleaner'> as a component component=<class 'haystack.components.preprocessors.document_cleaner.DocumentCleaner'> lineno=493 module=haystack.core.component.component
DEBUG:haystack.core.component.component:Registering <class 'haystack.components.preprocessors.document_cleaner.DocumentCleaner'> as a component
2025-04-09T09:20:00.081499Z [debug    ] Registered Component <class 'haystack.components.preprocessors.document_cleaner.DocumentCleaner'> component=<class 'haystack.components.preprocessors.document_cleaner.DocumentCleaner'> lineno=529 module=haystack.core.component.component
DEBUG:haystack.core.component.component:Registered Component <class 'haystack.components.preprocessors.document_cleaner.DocumentCleaner'>
2025-04-09T09:20:00.095007Z [debug    ] Registering <class 'haystack.components.preprocessors.document_splitter.DocumentSplitter'> as a component component=<c

 | node: Climate change
 |- node: Terminology
 |- node: Global temperature rise
 |-- node: Temperatures prior to present-day global warming
 |-- node: Warming since the Industrial Revolution
 |--- node: Differences by region
 >-- 1 summaries for section: Warming since the Industrial Revolution
 |-- node: Future global temperatures
 >- 3 summaries for section: Global temperature rise
 |- node: Causes of recent global temperature rise
 |-- node: Greenhouse gases
 |-- node: Land surface changes
 |-- node: Other factors
 |--- node: Aerosols and clouds
 |--- node: Solar and volcanic activity
 |--- node: Climate change feedbacks
 >-- 3 summaries for section: Other factors
 >- 3 summaries for section: Causes of recent global temperature rise
 |- node: Modelling
 |- node: Impacts
 |-- node: Environmental effects
 |-- node: Tipping points and long-term impacts
 |-- node: Nature and wildlife
 |-- node: Humans
 |--- node: Health and food
 |--- node: Livelihoods and inequality
 |--- node: Climate 

2025-04-09T09:20:03.889952Z [debug    ] receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Wed, 09 Apr 2025 09:20:03 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'klikkikuri'), (b'openai-processing-ms', b'1645'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'5000'), (b'x-ratelimit-limit-tokens', b'4000000'), (b'x-ratelimit-remaining-requests', b'4998'), (b'x-ratelimit-remaining-tokens', b'3998210'), (b'x-ratelimit-reset-requests', b'16ms'), (b'x-ratelimit-reset-tokens', b'26ms'), (b'x-request-id', b'req_62234baeb124d6d38e5b1ca2b2fd87e7'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'cf-cache-status', b'DYNAMIC'), (b'Set-Cookie', b'__cf_bm=6sss6R2Cc4a2ZwjCHwls_iEnpqi_p3itzWiuWFt9cyc-1744190403-1.0.1.1-8jeh8BonXQhZlFH3sqQTupeued16Rl72P0o8XSUY1e5

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /app/.venv/lib/python3.12/site-packages/httpx/_transports/default.py:101 in                      │
│ map_httpcore_exceptions                                                                          │
│                                                                                                  │
│    98 │   if len(HTTPCORE_EXC_MAP) == 0:                                                         │
│    99 │   │   HTTPCORE_EXC_MAP = _load_httpcore_exceptions()                                     │
│   100 │   try:                                                                                   │
│ ❱ 101 │   │   yield                                                                              │
│   102 │   except Exception as exc:                                                               │
│   103 │   │   mapped_exc = None                                                                  │
│   104                                                                                            │
│                                                                                                  │
│ ╭───────────────── locals ─────────────────╮                                                     │
│ │ message = 'The read operation timed out' │                                                     │
│ ╰──────────────────────────────────────────╯                                                     │
│                                                                                                  │
│ /app/.venv/lib/python3.12/site-packages/httpx/_transports/default.py:250 in handle_request       │
│                                                                                                  │
│   247 │   │   │   extensions=request.extensions,                                                 │
│   248 │   │   )                                                                                  │
│   249 │   │   with map_httpcore_exceptions():                                                    │
│ ❱ 250 │   │   │   resp = self._pool.handle_request(req)                                          │
│   251 │   │                                                                                      │
│   252 │   │   assert isinstance(resp.stream, typing.Iterable)                                    │
│   253                                                                                            │
│                                                                                                  │
│ ╭─────────────────────────────────────────── locals ───────────────────────────────────────────╮ │
│ │ httpcore = <module 'httpcore' from                                                           │ │
│ │            '/app/.venv/lib/python3.12/site-packages/httpcore/__init__.py'>                   │ │
│ │      req = <Request [b'POST']>                                                               │ │
│ │  request = <Request('POST', 'https://api.openai.com/v1/chat/completions')>                   │ │
│ │     self = <httpx.HTTPTransport object at 0x7ffb7403c350>                                    │ │
│ ╰──────────────────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                                  │
│ /app/.venv/lib/python3.12/site-packages/httpcore/_sync/connection_pool.py:256 in handle_request  │
│                                                                                                  │
│   253 │   │   │   │   closing = self._assign_requests_to_connections()                           │
│   254 │   │   │                                                                                  │
│   255 │   │   │   self._close_connections(closing)                                               │
│ ❱ 256 │   │   │   raise exc from None                      

2025-04-09T09:21:14.904783Z [debug    ] Encountered httpx.TimeoutException lineno=1002 module=openai._base_client

DEBUG:openai._base_client:Encountered httpx.TimeoutException
Traceback (most recent call last):
  File "/app/.venv/lib/python3.12/site-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    yield
  File "/app/.venv/lib/python3.12/site-packages/httpx/_transports/default.py", line 250, in handle_request
    resp = self._pool.handle_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/app/.venv/lib/python3.12/site-packages/httpcore/_sync/connection_pool.py", line 256, in handle_request
    raise exc from None
  File "/app/.venv/lib/python3.12/site-packages/httpcore/_sync/connection_pool.py", line 236, in handle_request
    response = connection.handle_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/app/.venv/lib/python3.12/site-packages/httpcore/_sync/connection.py", line 103, in handle_request
    return self._connection.handl

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /app/.venv/lib/python3.12/site-packages/httpx/_transports/default.py:101 in                      │
│ map_httpcore_exceptions                                                                          │
│                                                                                                  │
│    98 │   if len(HTTPCORE_EXC_MAP) == 0:                                                         │
│    99 │   │   HTTPCORE_EXC_MAP = _load_httpcore_exceptions()                                     │
│   100 │   try:                                                                                   │
│ ❱ 101 │   │   yield                                                                              │
│   102 │   except Exception as exc:                                                               │
│   103 │   │   mapped_exc = None                                                                  │
│   104                                                                                            │
│                                                                                                  │
│ ╭───────────────── locals ─────────────────╮                                                     │
│ │ message = 'The read operation timed out' │                                                     │
│ ╰──────────────────────────────────────────╯                                                     │
│                                                                                                  │
│ /app/.venv/lib/python3.12/site-packages/httpx/_transports/default.py:250 in handle_request       │
│                                                                                                  │
│   247 │   │   │   extensions=request.extensions,                                                 │
│   248 │   │   )                                                                                  │
│   249 │   │   with map_httpcore_exceptions():                                                    │
│ ❱ 250 │   │   │   resp = self._pool.handle_request(req)                                          │
│   251 │   │                                                                                      │
│   252 │   │   assert isinstance(resp.stream, typing.Iterable)                                    │
│   253                                                                                            │
│                                                                                                  │
│ ╭─────────────────────────────────────────── locals ───────────────────────────────────────────╮ │
│ │ httpcore = <module 'httpcore' from                                                           │ │
│ │            '/app/.venv/lib/python3.12/site-packages/httpcore/__init__.py'>                   │ │
│ │      req = <Request [b'POST']>                                                               │ │
│ │  request = <Request('POST', 'https://api.openai.com/v1/chat/completions')>                   │ │
│ │     self = <httpx.HTTPTransport object at 0x7ffb7403c350>                                    │ │
│ ╰──────────────────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                                  │
│ /app/.venv/lib/python3.12/site-packages/httpcore/_sync/connection_pool.py:256 in handle_request  │
│                                                                                                  │
│   253 │   │   │   │   closing = self._assign_requests_to_connections()                           │
│   254 │   │   │                                                                                  │
│   255 │   │   │   self._close_connections(closing)                                               │
│ ❱ 256 │   │   │   raise exc from None                      

2025-04-09T09:21:46.588734Z [debug    ] Encountered httpx.TimeoutException lineno=1002 module=openai._base_client

DEBUG:openai._base_client:Encountered httpx.TimeoutException
Traceback (most recent call last):
  File "/app/.venv/lib/python3.12/site-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    yield
  File "/app/.venv/lib/python3.12/site-packages/httpx/_transports/default.py", line 250, in handle_request
    resp = self._pool.handle_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/app/.venv/lib/python3.12/site-packages/httpcore/_sync/connection_pool.py", line 256, in handle_request
    raise exc from None
  File "/app/.venv/lib/python3.12/site-packages/httpcore/_sync/connection_pool.py", line 236, in handle_request
    response = connection.handle_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/app/.venv/lib/python3.12/site-packages/httpcore/_sync/connection.py", line 103, in handle_request
    return self._connection.handl

{'level': 1,
 'title': 'Climate change',
 'summary': "**Climate change** refers to the human-induced rise in global temperatures, primarily driven by fossil fuel burning, deforestation, and certain agricultural and industrial practices that release greenhouse gases. Earth's average surface air temperature has increased nearly 1.5 °C since the Industrial Revolution, with the Arctic experiencing the most significant warming. The impacts of climate change include expanding deserts, more frequent heat waves and wildfires, thawing permafrost, glacier retreat, and sea ice decline. Higher temperatures lead to intense storms, droughts, and other weather extremes, forcing species to relocate or face extinction. Climate change poses threats to human health, food and water security, and can lead to increased flooding, extreme heat, and economic loss. The World Health Organization identifies climate change as a major global health threat. Without action to limit warming, societies and ecosystems w

**Climate change** refers to the human-induced rise in global temperatures, primarily driven by fossil fuel burning, deforestation, and certain agricultural and industrial practices that release greenhouse gases. Earth's average surface air temperature has increased nearly 1.5 °C since the Industrial Revolution, with the Arctic experiencing the most significant warming. The impacts of climate change include expanding deserts, more frequent heat waves and wildfires, thawing permafrost, glacier retreat, and sea ice decline. Higher temperatures lead to intense storms, droughts, and other weather extremes, forcing species to relocate or face extinction. Climate change poses threats to human health, food and water security, and can lead to increased flooding, extreme heat, and economic loss. The World Health Organization identifies climate change as a major global health threat. Without action to limit warming, societies and ecosystems will face severe risks. The 2015 Paris Agreement aims to keep global warming well under 2 °C, but current pledges may lead to a rise of about 2.8 °C by century's end. Achieving a limit of 1.5 °C requires halving emissions by 2030 and reaching net-zero emissions by 2050. Transitioning to renewable energy sources and enhancing carbon capture methods are essential for mitigating climate change.